## Sentinel-1 Vessel Detections

Sentinel-1 synthetic aperture radar (SAR) scenes are accessed and processed through Google Earth Engine API. Vessel detections from January 1, 2017 through December 5, 2018 are processed and exported through 2 paths. First, as exports to Google Earth Engine Assets for visualization purposes that produces a single feature collection of vessels for each unique day (59 files). Secondly, as GeoJSON exports to a selected Google Drive folder (226 files) that produces a single GeoJSON of detected vessels for each unique scene.

In [45]:
import csv
import pandas as pd
from datetime import timedelta
from datetime import datetime
import ee
import math

In [46]:
ee.Initialize()

### Define Functions

In [47]:
def get_date(i):
    date = i.split("_")[4][0:8]
    return datetime.strptime(date,"%Y%m%d")

In [48]:
def extract_vessels_pre_s1_update (i):
    
    image = i.clip(i.geometry().buffer(-100)) # remove edge detections

    # Incidence angle normalization
    angleCorrected = (image.select('V.|H.')
                                .subtract(image
                                .select('angle')
                                .multiply(math.pi/180.0).pow(2).cos().log10().multiply(10.0)))
    
    # Remove SAR border boise
    validPixels = i.select('VH').gte(-45)
    borderNoiseRmvd = i.mask(validPixels).mask(water_raster_1km.eq(1))
    t = borderNoiseRmvd.select('VH').reduce(ee.Reducer.allNonZero())
    u = t.addBands(t).select(['all', 'all_1']).eq(1)
    v = u.mask(u)

    # Transform image from decibels to digital number (dn):
    dn = ee.Image(10.0).pow(angleCorrected.divide(10.0)).multiply(10000).mask(water_raster_1km).mask(v).select('VH')

    # Focal median & image differencing
    dn_vh_median = dn.focal_median(radius = 250, kernelType = 'circle', units = 'meters')
    dn_vh_diff = dn.subtract(dn_vh_median)

    # Threshold to isolate candidate pixels based on a quick survey of pixel distributions.
    threshold = dn_vh_median.multiply(ee.Image(6))
    candidatePixels = dn_vh_diff.select(0).gte(threshold).addBands(dn_vh_diff.gte(threshold))
    
    # Sieve
    connectedPixelsCt = candidatePixels.mask(candidatePixels).connectedPixelCount(4)
    sievedClusters = connectedPixelsCt.gte(2).mask(connectedPixelsCt.gte(2))
    
    # Erosion & Dilation
    candidatePixelsCleaned = (sievedClusters
                                  .focal_min(radius= 10, kernelType='square', units= 'meters')
                                  .focal_max(radius= 70, kernelType='square', units= 'meters'))

    # Reduce each cluster of candidate pixels to its centroid:
    centroids = (candidatePixelsCleaned.reduceToVectors(
        reducer = ee.Reducer.first(),
        geometry = borderNoiseRmvd.geometry(),
        scale = 20,
        geometryType = 'centroid',
        eightConnected = True,
        bestEffort = False,
        maxPixels = 1e13))
    
    return centroids

In [49]:
def extract_vessels_post_s1_update (i):
    
    image = i.clip(i.geometry().buffer(-100)) # remove edge detections

    # Incidence angle normalization
    angleCorrected = (image.select('V.|H.')
                                .subtract(image
                                .select('angle')
                                .multiply(math.pi/180.0).pow(2).cos().log10().multiply(10.0)))
    
    # Remove SAR border boise
    validPixels = i.select('VH').gte(-45)
    borderNoiseRmvd = i.mask(validPixels).mask(water_raster_1km.eq(1))
    t = borderNoiseRmvd.select('VH').reduce(ee.Reducer.allNonZero())
    u = t.addBands(t).select(['all', 'all_1']).eq(1)
    v = u.mask(u)

    # Transform image from decibels to digital number (dn):
    dn = ee.Image(10.0).pow(angleCorrected.divide(10.0)).multiply(10000).mask(water_raster_1km).mask(v).select('VH')

    # Focal median & image differencing
    dn_vh_median = dn.focal_median(radius = 250, kernelType = 'circle', units = 'meters')
    dn_vh_diff = dn.subtract(dn_vh_median)

    # Threshold to isolate candidate pixels based on a quick survey of pixel distributions.
    threshold = dn_vh_median.multiply(ee.Image(15))
    candidatePixels = dn_vh_diff.select(0).gte(threshold).addBands(dn_vh_diff.gte(threshold))
    
    # Sieve
    connectedPixelsCt = candidatePixels.mask(candidatePixels).connectedPixelCount(4)
    sievedClusters = connectedPixelsCt.gte(2).mask(connectedPixelsCt.gte(2))
    
    # Erosion & Dilation
    candidatePixelsCleaned = (sievedClusters
                                  .focal_min(radius= 10, kernelType='square', units= 'meters')
                                  .focal_max(radius= 70, kernelType='square', units= 'meters'))

    # Reduce each cluster of candidate pixels to its centroid:
    centroids = (candidatePixelsCleaned.reduceToVectors(
        reducer = ee.Reducer.first(),
        geometry = borderNoiseRmvd.geometry(),
        scale = 20,
        geometryType = 'centroid',
        eightConnected = True,
        bestEffort = False,
        maxPixels = 1e13))
    
    return centroids

### Filter Sentinel-1 Imagery

In [50]:
# Set global variables

aoi = ee.Geometry.LineString([[130.4281240295404, 38.21035156274654], [131.44015189764673, 42.054202613551624]])
buffered_landWater_raster = ee.Image('users/christian/landWaterRaster_2018-05-10/global_landWater_buffers')
water_raster_1km = buffered_landWater_raster.select('b1')

In [51]:
# Filter to get list of all unique scenes to process

inputCollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
                   .filterBounds(aoi)
                   .filterDate('2017-01-01', '2018-12-31')
                   .filterMetadata('instrumentMode', 'equals', 'IW')
                   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                   .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                   .filter(ee.Filter.eq('resolution','H')))

imgs = ee.List(inputCollection.aggregate_array('system:index'))
df = pd.DataFrame(imgs.getInfo())
df

,0
0,S1A_IW_GRDH_1SDV_20180825T212233_20180825T2122...
1,S1A_IW_GRDH_1SDV_20180825T212258_20180825T2123...
2,S1B_IW_GRDH_1SDV_20170108T212133_20170108T2121...
3,S1B_IW_GRDH_1SDV_20170108T212158_20170108T2122...
4,S1B_IW_GRDH_1SDV_20170120T212132_20170120T2121...
5,S1B_IW_GRDH_1SDV_20170120T212157_20170120T2122...
6,S1B_IW_GRDH_1SDV_20170201T212132_20170201T2121...
7,S1B_IW_GRDH_1SDV_20170201T212157_20170201T2122...
8,S1B_IW_GRDH_1SDV_20170213T212132_20170213T2121...
9,S1B_IW_GRDH_1SDV_20170213T212157_20170213T2122...


In [52]:
# Collapse unique scenes into unique days to process

unique_days = []

for i in df[0]:
    date = i.split("_")[4][0:8]
    unique_days.append(date)
    
df_unique_days_temp = pd.DataFrame({"days": unique_days})
df_unique_days = df_unique_days_temp.days.unique()
df_unique_days = pd.DataFrame({"days":df_unique_days.astype(str)})
df_unique_days = df_unique_days.sort_values(['days'], ascending=True)
df_unique_days

,days
1,20170108
2,20170120
3,20170201
4,20170213
5,20170225
6,20170309
7,20170402
8,20170414
9,20170426
10,20170508


### Vessel Detections Part I: Exports to Google Earth Engine Assets

In [53]:
# Set Earth Engine Asset export filepath

filepath = "users/brianwong-gfw/prj-nk-squid/v20181121"

In [54]:
# For vessel detections before Sentinel-1 updates on March 13, 2018.
# Exports one feature collection asset per unique day.

for i in df_unique_days["days"][0:35]:

    date = datetime.strptime(i,"%Y%m%d")
    Y = date.year
    m = date.month
    d = date.day
    start_date = date+timedelta(days=-1)
    end_date = date+timedelta(days=1)
    print('Processing S1 Scenes for {}-{}-{}'.format(Y, m, d))
    
    inputCollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(start_date, end_date)
               .filterMetadata('instrumentMode', 'equals', 'IW')
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .filter(ee.Filter.eq('resolution','H')))
    
    centroids = inputCollection.map(extract_vessels_pre_s1_update).flatten()
    
    if m < 10:
        mm = "0"+str(m)
    else:
        mm = str(m)    
    if d < 10:
        dd = "0"+str(d)
    else:
        dd = str(d)
    
    task = ee.batch.Export.table.toAsset(
        collection = centroids,
        description = 'nk-vessel-detections-to-gee-{Y}{mm}{dd}'.format(Y=Y, mm=mm, dd=dd),
        assetId = filepath + "/{Y}{mm}{dd}".format(Y=Y, mm=mm, dd=dd))
    
    task.start()
    task_id = task.id
    print(task_id)
    task = ee.batch.Task(task_id)

Processing S1 Scenes for 2017-1-8
2JLSE3GDUIDRIZNDNWPGAN57
Processing S1 Scenes for 2017-1-20
2E553W4R37QROXZD23VJPMNB
Processing S1 Scenes for 2017-2-1
WOX55U444JGOXPF67ZJJU3V2
Processing S1 Scenes for 2017-2-13
X6KEQK2PQZ747QLAYHJF6UX6
Processing S1 Scenes for 2017-2-25
7RK72V4R4K43L7PGUGBCTCSL
Processing S1 Scenes for 2017-3-9
3UYDPA3JCFJ6L4TLFGLWTAST
Processing S1 Scenes for 2017-4-2
GRVICKDYTZGQZAPLE7AVH5SQ
Processing S1 Scenes for 2017-4-14
UKTKNCSUYH2DVNDAEDYVFJ7J
Processing S1 Scenes for 2017-4-26
PQEUADWM3VM777RQOFNUBK3Z
Processing S1 Scenes for 2017-5-8
6Y67TVHJ6WGOIPJWFDGJLZKU
Processing S1 Scenes for 2017-5-20
Z6IIL2MDTS2MGLAFNLA6VC5X
Processing S1 Scenes for 2017-6-1
JPYWNXKLSC3SVGYUN3GJTZ35
Processing S1 Scenes for 2017-6-13
GKFDGTWRI73WHH6EG3OYMVYA
Processing S1 Scenes for 2017-6-25
TT47OHRIOIJ5Y22XDUDVFXYR
Processing S1 Scenes for 2017-7-7
KXLQRRI6WXZLWDYUIQCAKIYL
Processing S1 Scenes for 2017-7-19
6PPW5IMS2BRTRM3V6KS7KVSI
Processing S1 Scenes for 2017-7-31
VKLIOX3AXUKE

In [55]:
# For vessel detections after Sentinel-1 updates on March 13, 2018.
# Exports one feature collection asset per unique day.

for i in df_unique_days["days"][35:]:

    date = datetime.strptime(i,"%Y%m%d")
    Y = date.year
    m = date.month
    d = date.day
    start_date = date+timedelta(days=-1)
    end_date = date+timedelta(days=1)
    print('Processing S1 Scenes for {}-{}-{}'.format(Y, m, d))
    
    inputCollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(start_date, end_date)
               .filterMetadata('instrumentMode', 'equals', 'IW')
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .filter(ee.Filter.eq('resolution','H')))
    
    centroids = inputCollection.map(extract_vessels_post_s1_update).flatten()
    
    if m < 10:
        mm = "0"+str(m)
    else:
        mm = str(m)    
    if d < 10:
        dd = "0"+str(d)
    else:
        dd = str(d)
    
    task = ee.batch.Export.table.toAsset(
        collection = centroids,
        description = 'nk-vessel-detections-to-gee-{Y}{mm}{dd}'.format(Y=Y, mm=mm, dd=dd),
        assetId = filepath + "/{Y}{mm}{dd}".format(Y=Y, mm=mm, dd=dd))
    
    task.start()
    task_id = task.id
    print(task_id)
    task = ee.batch.Task(task_id)

Processing S1 Scenes for 2018-3-16
V33UD625TOT44HQE6HZAKXQQ
Processing S1 Scenes for 2018-3-28
4Y6SVUAXTM3KW3EMWG3T5VIS
Processing S1 Scenes for 2018-4-9
COV4JPCYGRIKC57UZ7NNGHSY
Processing S1 Scenes for 2018-4-21
XMCWQGB6RBA7HRWOC5HS6X2V
Processing S1 Scenes for 2018-5-3
W7NK7K5LLIXPMKNN2P4AGR2I
Processing S1 Scenes for 2018-5-15
5HVCSHAFMQPVDELP6WIOLFTP
Processing S1 Scenes for 2018-5-27
FPN6YQ4BB573YBTTUSM2HDGR
Processing S1 Scenes for 2018-6-8
A3Q4LOXSVNGHQIWBLDCH2HIT
Processing S1 Scenes for 2018-6-20
6MZQ6TWCSTUB4O7WZASD5DJV
Processing S1 Scenes for 2018-7-2
5WWGTCTUCC7QNVSTIG43X3T2
Processing S1 Scenes for 2018-7-14
TGTN7NYOP2EV7D7BW65CXYWV
Processing S1 Scenes for 2018-7-26
LDVECP7R4M4KH4J745OOR5LR
Processing S1 Scenes for 2018-8-7
ZNVOFI3WUKD4IPR7WQXR5XRT
Processing S1 Scenes for 2018-8-19
3XUT6ZFJZ2346XYOPGVAMKEB
Processing S1 Scenes for 2018-8-25
4GXIGYKXVT3EVFVS2JQR77MF
Processing S1 Scenes for 2018-8-31
LG7MRHNFE66PNPC5PG26RPQW
Processing S1 Scenes for 2018-9-12
TMFYQNVN2E

### Vessel Detections Part II: GeoJSON Exports

In [65]:
# Set preferred Google Drive folder destination for GeoJSON exports from each scene
gdrive_folder = 'v20181121'

In [66]:
# GeoJSON exports for vessel detections before Sentinel-1 updates on March 13, 2018.
# Exports one GeoJSON per unique scene ID.

for i in df_unique_days["days"][0:35]:
    
    date = datetime.strptime(i,"%Y%m%d")
    Y = date.year
    m = date.month
    d = date.day
    start_date = date+timedelta(days=-1)
    end_date = date+timedelta(days=1)
    
    inputCollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(start_date, end_date)
               .filterMetadata('instrumentMode', 'equals', 'IW')
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .filter(ee.Filter.eq('resolution','H'))
               .toList(1000))
    
    for img in inputCollection.getInfo():
        
        img_string = img.get('properties').get('system:index')  
        scene = ee.Image('COPERNICUS/S1_GRD/' + img_string)

        centroids = (extract_vessels_pre_s1_update(scene))
        print('Processing S1 Scene ID: {}'.format(img_string))
        
        # Export to Drive as GeoJSON
        task = ee.batch.Export.table.toDrive(
            collection = centroids,
            description = 'nk-vessel-detections-to-gdrive-{img_string}'.format(img_string=img_string),
            folder = gdrive_folder,
            fileFormat = 'GeoJSON',
            fileNamePrefix = img_string)
    
        task.start()
        task_id = task.id
        print('GEE Task ID: {}'.format(task_id))
        task = ee.batch.Task(task_id)

Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170108T212133_20170108T212158_003762_00677A_39C8
GEE Task ID: OAWYNUEXUCAP5ERQZPGSXYIT
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170108T212158_20170108T212209_003762_00677A_3EBD
GEE Task ID: BQHIBEKF4IDRLWYJOPNN2MTW
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170120T212132_20170120T212157_003937_006C9E_25FD
GEE Task ID: MA75SRPH7LK4ILIBV2G6YFRU
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170120T212157_20170120T212209_003937_006C9E_36C7
GEE Task ID: JRHWAGRIG5CZXPLP3I4H2SSN
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170201T212132_20170201T212157_004112_0071D3_C5CF
GEE Task ID: 6G776EXYEG3Q7AOOZ77VTKTM
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170201T212157_20170201T212209_004112_0071D3_D454
GEE Task ID: VPEV6SXD33Y32KXLWP3HY3LF
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170213T212132_20170213T212157_004287_00770A_A7B0
GEE Task ID: RF4QAHRS774QTQ4F6WSWK3ZB
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170213T212157_20170213T212209_004287_00770A_B80

GEE Task ID: KM7JH6J45MC2ZTOBVES3JHJO
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170824T212140_20170824T212205_007087_00C7D0_AADA
GEE Task ID: NIQAHEH2ITYDKAN7EPWD35W5
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170824T212205_20170824T212230_007087_00C7D0_1807
GEE Task ID: VDMFZKPH7XZWUXD6UAQHBEXT
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170824T212230_20170824T212255_007087_00C7D0_CD54
GEE Task ID: R7DTIY2GCLPJRHJED6QHWSRG
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170824T212255_20170824T212320_007087_00C7D0_B552
GEE Task ID: HUM6IC62UQMQ3PU6BBC536UI
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170905T212140_20170905T212205_007262_00CCE6_F46D
GEE Task ID: 2ER7BFQAKFQHSCTB75KQBIUU
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170905T212205_20170905T212230_007262_00CCE6_E37B
GEE Task ID: MJYNFMMA52PEMTXII5LBZU7X
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170905T212230_20170905T212255_007262_00CCE6_6356
GEE Task ID: 4NJO6IXAZVMQ5DZW5G63KSWF
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20170905T21

GEE Task ID: QTDV3BMC4SQAWBORVHIVAQZR
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180304T212138_20180304T212203_009887_011E48_FC09
GEE Task ID: OZLTIWHPNB2K6D5YHCF6INEZ
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180304T212203_20180304T212228_009887_011E48_96A2
GEE Task ID: VHEGDYTB6NEJGNFVEBBQKPPS
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180304T212228_20180304T212253_009887_011E48_0FC4
GEE Task ID: 5LP3WAL7DXGASAP3IRMJICIS
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180304T212253_20180304T212318_009887_011E48_67B3
GEE Task ID: 4WTK7POSGDSGKFDYAUVZR2CD


In [67]:
# GeoJSON exports for vessel detections before Sentinel-1 updates on March 13, 2018.
# Exports one GeoJSON per unique scene ID.

for i in df_unique_days["days"][35:]:
    
    date = datetime.strptime(i,"%Y%m%d")
    Y = date.year
    m = date.month
    d = date.day
    start_date = date+timedelta(days=-1)
    end_date = date+timedelta(days=1)
    
    inputCollection = (ee.ImageCollection('COPERNICUS/S1_GRD')
               .filterBounds(aoi)
               .filterDate(start_date, end_date)
               .filterMetadata('instrumentMode', 'equals', 'IW')
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
               .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
               .filter(ee.Filter.eq('resolution','H'))
               .toList(1000))
    
    for img in inputCollection.getInfo():
        
        img_string = img.get('properties').get('system:index')  
        scene = ee.Image('COPERNICUS/S1_GRD/' + img_string)

        centroids = (extract_vessels_post_s1_update(scene))
        print('Processing S1 Scene ID: {}'.format(img_string))
        
        # Export to Drive as GeoJSON
        task = ee.batch.Export.table.toDrive(
            collection = centroids,
            description = 'nk-vessel-detections-to-gdrive-{img_string}'.format(img_string=img_string),
            folder = gdrive_folder,
            fileFormat = 'GeoJSON',
            fileNamePrefix = img_string)
    
        task.start()
        task_id = task.id
        print('GEE Task ID: {}'.format(task_id))
        task = ee.batch.Task(task_id)

Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180316T212138_20180316T212203_010062_01240D_5D54
GEE Task ID: CAY4TS2BNU7HOB22KPB4ZX5B
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180316T212203_20180316T212228_010062_01240D_4A71
GEE Task ID: KI26CEPCXPOBMOHTJQN7VDKC
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180316T212228_20180316T212253_010062_01240D_C002
GEE Task ID: W3OTW5Y73GVTCXG5KEZX2GYF
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180316T212253_20180316T212318_010062_01240D_FE65
GEE Task ID: FKOFPRUT6ST4XPH3NFCY3UN7
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180328T212139_20180328T212204_010237_0129BB_1EA0
GEE Task ID: 2LX6JKZESKWKFGH5SJ7STALH
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180328T212204_20180328T212229_010237_0129BB_8D68
GEE Task ID: SUUF6JTB7XYFTPGDEYI3NBWH
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180328T212229_20180328T212254_010237_0129BB_66C7
GEE Task ID: Y3YYSDAKWJBWC4QQMP43WPCO
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180328T212254_20180328T212319_010237_0129BB_F0F

GEE Task ID: N7A42T7L3YTSLLB3RURTMVXB
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180912T212237_20180912T212302_012687_01769D_BF29
GEE Task ID: BVULO2IPYP7M7YMMJHCHRO6U
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180912T212302_20180912T212327_012687_01769D_CB14
GEE Task ID: CV2QADHBWLNQWZEQKMB2ZPLT
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180924T212148_20180924T212213_012862_017C01_A260
GEE Task ID: EWMTF5EESRIWUAK7B7LCCB6Q
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180924T212213_20180924T212238_012862_017C01_E4D5
GEE Task ID: LVEDFMATYJJDS3FBBVG4YNJB
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180924T212238_20180924T212303_012862_017C01_C18D
GEE Task ID: SYDTVCGH5V77VIYQCV2ERCXB
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20180924T212303_20180924T212328_012862_017C01_A252
GEE Task ID: 7KHUG673MXEH7OXLPTKNWCEL
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20181006T212148_20181006T212213_013037_018158_9B1A
GEE Task ID: F5S7H3XILAB7LQVFADPDTLOR
Processing S1 Scene ID: S1B_IW_GRDH_1SDV_20181006T21